In [35]:
import json
import csv
import numpy as np
import argparse
import pandas as pd

from impyute.imputation.cs import fast_knn
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier

In [36]:
config = 'column_config.json'
data_input = 'Fireball_And_Bolide_Reports.csv'
data_output = 'Fireball_And_Bolide_Reports.csv'
input_path = 'input\\'
tmp_path = 'tmp\\'
output_path = 'output\\'

In [37]:
def build_argsparser():
    parser = argparse.ArgumentParser(description='X TEAM data filler')
    parser.add_argument('--input', type=str, default='Fireball_And_Bolide_Reports.csv', help='123')
    parser.add_argument('--output', type=str, default='Fireball_And_Bolide_Reports.csv', help='234')
    parser.add_argument('--strategy', type=str, default='knn', help='345')
    parser.add_argument('--config', type=str, default='column_config.json', help='456')
    return parser

In [38]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [39]:
# opt = build_argsparser().parse_args()
# data_input = opt.input
# data_output = opt.output

In [40]:
print('Reading input data from {}'.format(data_input))
header = []
column_type = [] # 0 for class, 1 for digit
train_digit = []
train_class = []
origin = []
# assume that the csv file contains at least 1 row and all features in 1st line are filled
with open(tmp_path+data_input) as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    header = next(reader, None)
    column_type = [0] * len(header)
    
    row = next(reader, None)
    for i, v in enumerate(row):
        if isfloat(v):
            column_type[i] = 1
    # 1st row is skipped
    
    for row in reader:
        row_digit = [float(v) if v != '' else np.nan for i, v in enumerate(row) if column_type[i] == 1]
        row_class = [v if v != '' else np.nan for i,v in enumerate(row) if column_type[i] == 0]
        row = [i if i != '' else np.nan for i in row]
        train_digit.append(row_digit)
        train_class.append(row_class)
        origin.append(row)
train_nd = np.asarray(train_digit)

Reading input data from Fireball_And_Bolide_Reports.csv


In [41]:
result = fast_knn(train_nd, k=5)
filled = []
for i, row in enumerate(train_class):
    filled.append(row+list(result[i]))
filled = np.asarray(train)

In [61]:
df = pd.DataFrame(filled)
for i in range(3, 10):
    df[i] = df[i].astype('float')
df.head(10)

,0,1,2,3,4,5,6,7,8,9
0,01/09/2015 10:41:11 AM,2.0N,28.8E,36.000000,17.245098,-10.700000,-7.600000,11.600000,1.390000e+11,0.41
1,05/16/2014 12:42:48 PM,44.2S,176.2W,44.000000,16.077517,14.400000,4.600000,6.500000,3.090000e+11,0.82
2,08/23/2014 06:29:41 AM,61.7S,132.6E,22.200000,16.200000,-2.300000,5.700000,16.500000,3.820000e+12,7.60
3,12/12/2014 06:48:11 AM,33.5N,144.9E,26.300000,17.083333,11.500000,-2.800000,-2.200000,3.300000e+10,0.11
4,07/29/2014 07:38:07 AM,49.2S,172.2W,38.015385,17.083333,3.210709,-9.212217,-0.725490,7.300000e+10,0.23
5,06/01/2013 10:49:48 PM,65.6S,138.4E,28.900000,17.083333,6.469608,-2.679412,-1.415686,3.400000e+10,0.12
6,02/17/2015 01:19:50 PM,8.0S,11.2W,39.000000,17.083333,-28.200000,3.400000,4.600000,3.300000e+10,0.11
7,10/17/2014 02:07:36 PM,4.6S,66.3W,39.000000,17.083333,0.685383,-12.478075,-6.093761,7.200000e+10,0.23
8,12/08/2013 03:10:09 AM,32.8N,165.1W,23.500000,17.083333,2.300000,2.500000,-11.300000,6.400000e+10,0.20
9,09/05/2014 09:37:26 PM,22.7N,150.0W,28.859950,17.083333,4.792810,-2.639216,-1.154248,3.500000e+10,0.12
